In [ ]:
import pandas as pd
import stanza

#Stanza pipeline suomen kieltä varten
nlp = stanza.Pipeline('fi', processors='tokenize,pos,lemma', use_gpu=False)

#Funktio tekstin lemmatisointia varten
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_words = [word.lemma for sent in doc.sentences for word in sent.words]
    return ' '.join(lemmatized_words)

# Aineisto
df = pd.read_csv('arvostelut.csv', delimiter=';', quoting=0)
print(df.head())

#Lemmatisoinnin sovellus
df['lemmatized_text'] = df['teksti'].apply(lemmatize_text)
#print(df[['teksti', 'lemmatized_text']].head())
# Syöte X, data jonka avulla malli oppii. Kohde y, se mitä mallin tulee oppia ennustamaan.
X = df['lemmatized_text']
#X = df['teksti']
y = df['tunne']

# train_test_split sekoittaa datan ja jakaa sen neljään osaan: X_train ja y_train opetusdata, X_test ja y_test testidata
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=77, stratify=y
)

# Pipeline: TF-IDF + Logistic Regression
from sklearn.pipeline import Pipeline #ketjuttaa datan muokkausvaiheet ja malli yhteen pakettiin
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()), #vastaanottaa tekstin ja muuttaa sen numeroiksi TF-IDF:n avulla
    ('clf', LogisticRegression(class_weight='balanced')), #Luokittelija, ottaa vastaan TfidVectorizer:n tuottaman numerodatan ja tekee ennusteita sen perusteella. 
    #('clf', MultinomialNB()),
    #('clf', LinearSVC(class_weight='balanced', dual="auto")),
])


text_clf.fit(X_train, y_train) #.fit() komento syöttää opetusdatan X_train ja y_train putken läpi, TfidVectorizer analysoi X_trainin sanat ja luo sanaston sekä laskee TF-IDF arvot.

#Numerodata ja vastaukset y_train syötetään mallille joka oppii mitkä sanat liittyvät mihinkin tunteeseen
from sklearn.metrics import classification_report, accuracy_score
predictions = text_clf.predict(X_test) #Ennusteet testidatalla
#Tulostetaan raportti
print(f"Accuracy: {accuracy_score(y_test, predictions)}")
print(classification_report(y_test, predictions))


2025-08-11 13:06:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-08-11 13:06:39 INFO: Downloaded file to C:\Users\huvif\stanza_resources\resources.json
2025-08-11 13:06:39 WARNING: Language fi package default expects mwt, which has been added
2025-08-11 13:06:39 INFO: Loading these models for language: fi (Finnish):
| Processor | Package      |
----------------------------
| tokenize  | tdt          |
| mwt       | tdt          |
| pos       | tdt_charlm   |
| lemma     | tdt_nocharlm |

2025-08-11 13:06:39 INFO: Using device: cpu
2025-08-11 13:06:39 INFO: Loading: tokenize
2025-08-11 13:06:39 INFO: Loading: mwt
2025-08-11 13:06:39 INFO: Loading: pos
2025-08-11 13:06:40 INFO: Loading: lemma
2025-08-11 13:06:41 INFO: Done loading processors!


  otsikko                                             teksti         tunne  \
0     NaN  Ostin hyvästä tarjouksesta pesukoneen asennuks...  positiivinen   
1     NaN  Ensimmäiset viikot pyykätty ja tuntuisi olevan...  positiivinen   
2     NaN  Onhan näissä koneissa eroa kun ostaa 300€ kone...  positiivinen   
3     NaN  Vasta kaksi kertaa käytetty: mix -ohjelma ja l...  positiivinen   
4     NaN  Hommattiin pari viikkoa sitten ja oikeastaan s...  positiivinen   

                                                 url  tähdet?  
0  https://www.gigantti.fi/product/kodinkoneet/py...  5.touko  
1  https://www.gigantti.fi/product/kodinkoneet/py...  4.touko  
2  https://www.gigantti.fi/product/kodinkoneet/py...  5.touko  
3  https://www.gigantti.fi/product/kodinkoneet/py...  4.touko  
4  https://www.gigantti.fi/product/kodinkoneet/py...  5.touko  
                                              teksti  \
0  Ostin hyvästä tarjouksesta pesukoneen asennuks...   
1  Ensimmäiset viikot pyykätty ja t